# 分子数据转换为图结构

## 1. 数据提取

In [1]:
# 读取 QM9_demo 数据集
from utils.data import QM9DataExtractor

# 使用定义的QM9DataExtractor类用于加载提取数据
extractor = QM9DataExtractor('../data/demo/qm9_demo.csv')

**测试数据提取功能**

In [2]:
# 使用 SMILES 字符串 "C"（甲烷）测试数据提取功能
info = extractor.get_molecule_info('C')
print("数据结构:")
print("坐标数据keys:", info['coordinates_data'].keys())
print("属性数据keys:", info['properties_data'].keys())
info

数据结构:
坐标数据keys: dict_keys(['smiles', 'atoms', 'coordinates', 'charges', 'num_atoms'])
属性数据keys: dict_keys(['smiles', 'properties', 'property_vector'])


{'coordinates_data': {'smiles': 'C',
  'atoms': ['C', 'H', 'H', 'H', 'H'],
  'coordinates': array([[-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
         [ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03],
         [ 1.01173084e+00,  1.46375116e+00,  2.76574800e-04],
         [-5.40815069e-01,  1.44752661e+00, -8.76643715e-01],
         [-5.23813634e-01,  1.43793264e+00,  9.06397294e-01]]),
  'charges': array([-0.535689,  0.133921,  0.133922,  0.133923,  0.133923]),
  'num_atoms': 5},
 'properties_data': {'smiles': 'C',
  'properties': {'A': 157.7118,
   'B': 157.70997,
   'C': 157.70699,
   'miu': 0.0,
   'alpha': 13.21,
   'homo': -0.3877,
   'lumo': 0.1171,
   'gap': 0.5048,
   'R2': 35.3641,
   'zpve': 0.044749,
   'Uo': -40.47893,
   'U': -40.476062,
   'H': -40.475117,
   'G': -40.498597,
   'Cv': 6.469},
  'property_vector': array([ 1.5771180e+02,  1.5770997e+02,  1.5770699e+02,  0.0000000e+00,
          1.3210000e+01, -3.8770000e-01,  1.1710000e-01,  5.0480000e-01,
    

## 2. 图结构转换

**节点、边特征定义**
- 节点：
1. 原子类型（one-hot编码, H, C, N, O, F）
2. 原子坐标（x, y, z）
3. 原子电荷
4. 原子度数（RDkit）
5. 杂化类型（RDkit）

- 边：
1. 几何距离（键长）
2. 化学键类型（RDkit）
3. 共轭性（RDkit）
4. 立体化学信息（RDkit）

In [3]:
# 导入图转换器
from utils.data import MoleculeToGraph

# 创建图转换器实例
graph_converter = MoleculeToGraph()

# 使用甲烷分子测试
molecule_data = extractor.get_molecule_info('CC(=O)C')
smiles = 'CC(=O)C'

# 转换为图结构
graph_data = graph_converter.convert_to_graph(molecule_data, smiles)

print("图结构信息:")
print(f"节点数量: {graph_data['num_nodes']}")
print(f"边数量: {graph_data['num_edges']}")
print(f"节点特征维度: {graph_data['x'].shape}")
print(f"边索引形状: {graph_data['edge_index'].shape}")
print(f"边特征形状: {graph_data['edge_attr'].shape}")

print("\n节点特征详情:")
print("特征组成: 原子类型(5) + 坐标(3) + 电荷(1) + 度数(1) + 杂化类型(7) = 17维")
print("节点特征矩阵:")
print(graph_data['x'])

图结构信息:
节点数量: 10
边数量: 18
节点特征维度: torch.Size([10, 17])
边索引形状: torch.Size([2, 18])
边特征形状: torch.Size([18, 7])

节点特征详情:
特征组成: 原子类型(5) + 坐标(3) + 电荷(1) + 度数(1) + 杂化类型(7) = 17维
节点特征矩阵:
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -3.1019e-03,  1.4805e+00, -1.7260e-01, -4.6869e-01,  4.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -4.5272e-02, -2.8871e-02,  1.3949e-03,  3.9152e-01,  3.0000e+00,
          0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.2971e+00, -7.2194e-01,  1.6889e-01, -4.6869e-01,  1.0000e+00,
          0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, 

In [4]:
print("边特征详情:")
print("特征组成: 几何距离(1) + 化学键类型(4) + 共轭性(1) + 立体化学(1) = 7维")
print("边索引 (源节点 -> 目标节点):")
print(graph_data['edge_index'].T)
print("\n边特征矩阵:")
print(graph_data['edge_attr'])

print("\n节点特征分解 (以第一个节点为例):")
node_0_features = graph_data['x'][0]
print(f"原子类型 (one-hot): {node_0_features[:5].numpy()}")
print(f"坐标 (x,y,z): {node_0_features[5:8].numpy()}")
print(f"电荷: {node_0_features[8].numpy()}")
print(f"度数: {node_0_features[9].numpy()}")
print(f"杂化类型 (one-hot): {node_0_features[10:17].numpy()}")

边特征详情:
特征组成: 几何距离(1) + 化学键类型(4) + 共轭性(1) + 立体化学(1) = 7维
边索引 (源节点 -> 目标节点):
tensor([[0, 1],
        [1, 0],
        [1, 2],
        [2, 1],
        [1, 3],
        [3, 1],
        [0, 4],
        [4, 0],
        [0, 5],
        [5, 0],
        [0, 6],
        [6, 0],
        [3, 7],
        [7, 3],
        [3, 8],
        [8, 3],
        [3, 9],
        [9, 3]])

边特征矩阵:
tensor([[1.5200, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.5200, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.5200, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.5200, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2087, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2087, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0960, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0960, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0905, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0905, 1.0000, 0.0000, 0

## 3. 特征提取总结

**节点特征 (17维):**
1. 原子类型: one-hot编码 (H, C, N, O, F) - 5维
2. 原子坐标: (x, y, z) - 3维  
3. 原子电荷: 来自QM9数据集 - 1维
4. 原子度数: 从RDKit获取 - 1维
5. 杂化类型: one-hot编码 - 7维

**边特征 (7维):**
1. 几何距离: 根据原子坐标计算 - 1维
2. 化学键类型: one-hot编码 (单键、双键、三键、芳香键) - 4维
3. 共轭性: 布尔值 - 1维  
4. 立体化学信息: 是否在环中 - 1维

**实现特点:**
- 结合QM9原始数据(坐标、电荷)和RDKit化学信息
- 支持仅使用坐标数据的回退模式
- 输出PyTorch Geometric兼容的数据格式